In [ ]:
import $file.common
import spark._
import common._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}

In [ ]:
import spark.implicits._
import spark.sqlContext.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark._
import org.apache.spark.sql.types._, func._
import org.apache.spark.sql.functions.{col, to_date}
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._


# Media de infecciones por Km2 

### Utilizando RDDs

In [ ]:
def infections(lines : RDD[String]) : RDD[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(
        day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10)
      )
    })

In [ ]:
val infectionRDD = infections(spark.sparkContext.textFile("../datasets/data.csv"))

In [ ]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Population(
    country : String, 
    population : Int, 
    density : Int, 
    land_area: Int, 
    ) 
extends Serializable

In [ ]:
val populationData = spark.sparkContext.textFile("../datasets/population_by_country_2020.csv")

In [ ]:
val header = populationData.first() 

def population(lines : RDD[String]) : RDD[Population] =
    lines.filter(x => x != header)
    .map(line => {
      val arr = line.split(",")
      Population(
        country = arr(0),
        population = arr(1).toInt,
        density = arr(4).toInt,
        land_area = arr(5).toInt,
      )
    })

In [ ]:
val populationRDD = population(populationData)

### Un join computacionalmente pesado desde el principio ya que cruza todos los datos sin quedarnos con los que nos interesen

Spark no me deja hacer un Join de RDD que no sean pair RDD así que tenemos que construirlo

In [ ]:
// populationRDD.join(infectionRDD)

Construyo Pair RDDs conservando todos los datos

In [ ]:
val populationByCountry = populationRDD.map(
    x => (x.country,x))

val infectionByCountry = 
      infectionRDD.map(x => (x.country,x))

Hago el Join y agrupo por paises

In [ ]:
val joinedRDD = infectionByCountry.join(populationByCountry).groupByKey()

Finalmente calculo la media

In [ ]:
joinedRDD.mapValues(
    x => x.map( 
        line => line._1.nCases.toFloat / line._2.land_area.toFloat
    )).mapValues(
    x => x.sum / x.size
).collect()

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [ ]:
val notOptimizedRDD =
    infectionByCountry.join(populationByCountry)
    .groupByKey()
    .mapValues(
    x => x.map( 
        line => line._1.nCases.toFloat / line._2.land_area.toFloat)
    ).mapValues(
        x => x.sum / x.size
    )

#### Para optimizar un poco esta consulta:

Despejo solo los datos que me interesan para trabajar con Pair RDDs y optimizar la consulta

In [ ]:
val countriesAndLandArea = populationRDD.map(
    x => (x.country,x.land_area))

In [ ]:
val countriesAndCases = 
      infectionRDD.map(x => (x.country,x.nCases))
      .groupByKey()

Ejecuto un join y trabajo para calcular primero la media de infecciones por Km2 diaria, 
para luego calcular la media total

In [ ]:
val average = countriesAndCases.join(countriesAndLandArea)

In [ ]:
average.mapValues(
    x => x._1.map(
        y => (y.toFloat / x._2.toFloat)
    )).mapValues(
    x => x.sum/x.size
).collect()

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [ ]:
val meanInfectionsRDD =
countriesAndCases.join(countriesAndLandArea)   
.mapValues(
    x => x._1.map(
        y => (y.toDouble / x._2.toDouble)
    )).mapValues(
    x => x.sum / x.size
)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    meanInfectionsRDD.collect
)

## Consulta con DataSet

In [ ]:
val infectionDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/covidworldwide.csv")
.withColumnRenamed("countriesAndTerritories","Country")
.as[(String,String,String,String,Double,Double,String,String,String,String,String,String)]

In [ ]:
val populationDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/population_by_country_2020.csv")
.withColumnRenamed("Country (or dependency)","Country")
.withColumnRenamed("Population (2020)","Population")
.as[(String,Float,String,Float,Float,Float,Double,String,String,String,String)]

In [ ]:
val meanInfectionsperKM2DS = 
infectionDS.join(populationDS, "Country")
        .select($"Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("Country")
        .agg(round(avg("infection Per Km\u00b2"),10).as[Float])
        .orderBy(desc("round(avg(infection Per Km²), 10)"))
        .as[(String,Double)]

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(meanInfectionsperKM2DS.collect)

## Consulta con DataFrame

In [ ]:
val dfCovid = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/covidworldwide.csv")

In [ ]:
val dfPopulation = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/population_by_country_2020.csv")
.withColumnRenamed("Country (or dependency)","Country")
.withColumnRenamed("Population (2020)","Population")
dfPopulation.showHTML()
dfPopulation.schema

Modifico los datos de entrada para que el formato fecha se adecue al TimeStamp de Spark

In [ ]:
val dfCovidClean = dfCovid
    .select($"*",$"dateRep",translate($"dateRep","/","-").as("date"))
    .drop("dateRep")

In [ ]:
val dfCovidDate = dfCovidClean
    .select($"*",col("date"),to_date(col("date"),"dd-MM-yyyy").as("to_date"))

Hago una consulta de prueba para obtener la media solo de los casos en España

In [ ]:
val spainCovid = dfCovid.select("dateRep","cases").where("countriesAndTerritories == 'Spain'").toDF

In [ ]:
spainCovid.agg(avg("cases")).showHTML()

### Finalmente ejecuto la consulta de nuestro caso de uso, infecciones por Km2

In [ ]:
val meanInfectionsperKM2DF = 
dfCovid.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("country")
        .avg("infection Per Km\u00b2")
        .orderBy(desc("avg(infection Per Km²)"))

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    meanInfectionsperKM2DF.collect
    )

# Visualización de datos con plotly

In [ ]:
val (x,y) = meanInfectionsperKM2DF.collect.map(r=>(r(0).toString, r(1).toString.toFloat)).toList.unzip
Bar(x, y).plot()

# Visualización de eficiencia

In [ ]:
val (x, y) = Seq(
    "Not Optimized RDD" -> runWithOutput(notOptimizedRDD.collect),
    "RDD" -> runWithOutput(meanInfectionsRDD.collect),
    "DataSet" -> runWithOutput(meanInfectionsperKM2DS.collect),
    "DataFrame" -> runWithOutput(meanInfectionsperKM2DF.collect),
).unzip

Bar(x, y).plot()